# Autopacmen Workflow
This script collects the scripts from the Autopacmen workflow

In [7]:
# setup code: execute and continue below

# general imports
import pathlib
import os
import cobra
import sys

# enabling local imports of autopacmen scripts
sys.path.append(str(pathlib.Path(os.getcwd()).parent.parent / "autopacmen"))


# create folders for input & output data
current_dir = pathlib.Path(os.getcwd())
setup_dir = current_dir.parent / "datasets" / "sMOMENT" / "autopacmen_setup"
input_dir = current_dir.parent / "datasets" / "sMOMENT" / "autopacmen_input"
os.makedirs(setup_dir, exist_ok=True)
os.makedirs(input_dir, exist_ok=True)
os.makedirs(input_dir.parent / "autopacmen_output", exist_ok=True)

In [11]:
'''
In order for sMOMENT to work, your project folder should look like this:

project/
├── autopacmen/
│   ├── __init__.py
│   ├── ... .py
│   └── submodules/
│       ├── __init__.py
│       └── ... .py
├── thesis_scripts/
│   ├── models/
│   │   └── your_ftINIT_model.mat
│   └── sMOMENT/
│       └── autopacmen_workflow.ipynb


Before executing the scripts, there are a few necessary precautions:
-   download BIGG metabolites data from from http://bigg.ucsd.edu/data_access (last accessed: 09/12/2024) and 
    save it into "datasets/sMOMENT/autopacmen_setup/bigg_models_metabolites.txt"
-   download BRENDA database data from https://www.brenda-enzymes.org/download.php (last accessed: 09/12/2024) and 
    save it into "datasets/sMOMENT/autopacmen_setup/brenda_2023_1.txt"
-   input the name of your model created in ftINIT and the project name below:
'''
model_name = "A375_ftINIT_model.mat"
project_name = "A375"
output_model_name = "A375_sMOMENT_model.xml"




In [12]:
# get model directory
model_dir = current_dir.parent / "models" / model_name
xml_model_dir = model_dir.with_stem(model_dir.stem + "_prep").with_suffix(".xml")

# load matlab model, assign missing annotations and save as sbml model
if not xml_model_dir.is_file():
    model = cobra.io.load_matlab_model(model_dir)
    ihuman_xml = cobra.io.read_sbml_model(model_dir.parent / "Human-GEM.xml")
    # iterating over reactions in the model and assigning the respective annotations lost in the matlab representation
    for reaction in model.reactions:
        model.reactions.get_by_id(reaction.id).annotation.update(ihuman_xml.reactions.get_by_id(reaction.id).annotation)
    for gene in model.genes:
        model.genes.get_by_id(gene.id).annotation.update(ihuman_xml.genes.get_by_id(gene.id).annotation)

    cobra.io.write_sbml_model(model, xml_model_dir)

In [13]:
# run data preparation scripts 

# specifying the input directories
bigg_dir = setup_dir / "bigg_models_metabolites.txt"
brenda_dir = setup_dir / "brenda_2023_1.txt"

# converting BIGG metabolites file to JSON
# inputs:   - path to the "bigg_models_metabolites.txt" file (as string)
#           - path to the folder where the JSON file should be saved in (as string)
# outputs:  - JSON file named "bigg_id_name_mapping.json"
# BIGG metabolites data from downloaded from http://bigg.ucsd.edu/data_access (accessed: 09/12/2024)
from autopacmen.submodules.parse_bigg_metabolites_file import parse_bigg_metabolites_file
parse_bigg_metabolites_file(
    str(bigg_dir), 
    str(setup_dir)
)

# converting sMOMENT relevant data from Brenda textfile to JSON
# inputs:   - path to the "brenda_2023_1.txt" file (as string)
#           - path to the "bigg_id_name_mapping.json" file generated in the previous script (as string)
#           - path to the file (not the folder!) where the JSON file should be saved in (as string)
# outputs:  - JSON file named "brenda.json" (if not stated otherwise in the inputs)
# Brenda database data downloaded from https://www.brenda-enzymes.org/download.php (accessed: 09/12/2024)
from autopacmen.submodules.parse_brenda_textfile import parse_brenda_textfile
parse_brenda_textfile(
    str(brenda_dir), 
    str(setup_dir), 
    str(setup_dir / "brenda.json")
)

# makes the previous "brenda.json" file specific for the model
# inputs:   - path to the model sbml file
#           - path to the previous "brenda.json" file (as string)
#           - path to output file (not folder)
# outputs:  - JSON file named "brenda_model_specific.json" (if not stated otherwise in the inputs)
from autopacmen.submodules.parse_brenda_json_for_model import parse_brenda_json_for_model
parse_brenda_json_for_model(
    str(xml_model_dir), 
    str(setup_dir / "brenda.json"), 
    str(setup_dir / "brenda_model_specific.json")
)



In [5]:
# maps kcat values derived from the SABIO-RK database to model EC-numbers, 
# allowing for "wildcards" (meaning using values from related proteins if no kcat values are available)
# inputs:   - path to the model sbml file
#           - path to output file (not folder)
#           - path to the previous "bigg_id_name_mapping.json" file (as string)
# outputs:  - JSON file named "sabio_rk.json" (if not stated otherwise in the inputs) (as string)
# SABIO-RK was accessed on 10/12/2024
from autopacmen.submodules.parse_sabio_rk_for_model import parse_sabio_rk_for_model_with_sbml
parse_sabio_rk_for_model_with_sbml(
    xml_model_dir, 
    str(setup_dir / "sabio_rk.json"), 
    str(setup_dir / "bigg_id_name_mapping.json")
)

KeyboardInterrupt: 

In [6]:
# combines kcat values mapped to EC-numbers from SABIO-RK and BRENDA into one json
# inputs:   - path to SABIO-RK json mapping kcat values to EC-Numbers of the reactions in the model (as string)
#           - path to brenda json kcat database (as string)
#           - path to the output file (not folder)
# output:   - combined json file named "sabio.json"
from autopacmen.submodules.create_combined_kcat_database import create_combined_kcat_database
create_combined_kcat_database(
    str(setup_dir / "sabio_rk.json"), 
    str(setup_dir / "brenda_model_specific.json"), 
    str(setup_dir / "sabio.json")
)

In [ ]:
# creates xlsx spreadsheets for the input information
# inputs:   - path to SBML model 
#           - path to the folder where the sheets should be created in
#           - project name
# outputs:  - xlsx spreadsheets to input data about the organism (only the enzyme_stoichiometries and protein_data sheets are essential)
from autopacmen.submodules.get_initial_spreadsheets import get_initial_spreadsheets_with_sbml
get_initial_spreadsheets_with_sbml(
    str(xml_model_dir), 
    str(input_dir), 
    str(project_name)
)

INFO: Reaction MAR08360 does not have a KEGG ID annotation
INFO: Reaction MAR05396 does not have a KEGG ID annotation
INFO: Reaction MAR09727 does not have a KEGG ID annotation
INFO: Reaction MAR05397 does not have a KEGG ID annotation
INFO: Reaction MAR05398 does not have a KEGG ID annotation
INFO: Reaction MAR05399 does not have a KEGG ID annotation
INFO: Reaction MAR05400 does not have a KEGG ID annotation
INFO: Reaction MAR05401 does not have a KEGG ID annotation
INFO: Reaction MAR08578 does not have a KEGG ID annotation
INFO: Reaction MAR08591 does not have a KEGG ID annotation
INFO: Reaction MAR08592 does not have a KEGG ID annotation
INFO: Reaction MAR04130 does not have a KEGG ID annotation
INFO: Reaction MAR08762 does not have a KEGG ID annotation
INFO: Reaction MAR08727 does not have a KEGG ID annotation
INFO: Reaction MAR08728 does not have a KEGG ID annotation
INFO: Reaction MAR08500 does not have a KEGG ID annotation
INFO: Reaction MAR08501 does not have a KEGG ID annotati

In [ ]:
# Creates a JSON with the protein masses from UniProt for all proteins given in the gene rules of the given metabolic model
# inputs:   - path to the model smbl file (as string)
#           - path to the project folder (as string)
#           - project name
# outputs:  - json file with the mapping called project name + "_protein_id_mass_mapping.json"

from autopacmen.submodules.get_protein_mass_mapping import get_protein_mass_mapping_with_sbml
get_protein_mass_mapping_with_sbml(
    str(xml_model_dir), 
    str(input_dir), 
    str(project_name)
)

{'O60762': ['ENSG00000000419'], 'Q9BTY2': ['ENSG00000001036'], 'P48506': ['ENSG00000001084'], 'Q16850': ['ENSG00000001630'], 'P28838': ['ENSG00000002549'], 'O14792': ['ENSG00000002587'], 'P19801': ['ENSG00000002726'], 'Q9NR63': ['ENSG00000003137'], 'Q9Y216': ['ENSG00000003987'], 'P52569': ['ENSG00000003989'], 'P54819': ['ENSG00000004455'], 'P28907': ['ENSG00000004468'], 'O14561': ['ENSG00000004779'], 'Q16654': ['ENSG00000004799'], 'Q86VW1': ['ENSG00000004809'], 'Q9UJS0': ['ENSG00000004864'], 'P02730': ['ENSG00000004939'], 'P05141': ['ENSG00000005022'], 'P52435': ['ENSG00000005075'], 'Q53FZ2': ['ENSG00000005187'], 'P05164': ['ENSG00000005381'], 'P27169': ['ENSG00000005421'], 'Q9UKG9': ['ENSG00000005469'], 'P21439': ['ENSG00000005471'], 'Q15119': ['ENSG00000005882'], 'Q09428': ['ENSG00000006071'], 'Q53H12': ['ENSG00000006530'], 'P43353': ['ENSG00000006534'], 'O75223': ['ENSG00000006625'], 'P51689': ['ENSG00000006756'], 'P41247': ['ENSG00000006757'], 'O95045': ['ENSG00000007001'], 'P43034

In [ ]:
# maps kcat values from BRENDA/SABIO RK databases as well as optional custom values to the reactions in the model
# inputs:   - path to the SBML of the metabolic model
#           - path to the output folder 
#           - project name
#           - scientific name of the organism (used for the taxonomy-dependent search of kcat values)
#           - path to the SABIO-RK&BRENDA kcat<->reaction mapping JSON (from data_create_combined_kcat_database.py)
#           - *optional* path to the custom user-defined kcat<->protein JSON (default: "")
#           - type of kcat selection TEXT Can be "mean", "median" or "random"
# output:   - json with kcat values mapped to the reactions

from autopacmen.submodules.get_reactions_kcat_mapping import get_reactions_kcat_mapping

get_reactions_kcat_mapping(
    sbml_path=str(xml_model_dir),
    project_folder=str(input_dir),
    project_name=str(project_name),
    organism='Homo sapiens',
    kcat_database_path=str(setup_dir / "sabio.json"),
    protein_kcat_database_path="",
    type_of_kcat_selection='mean'
)

# Applies the sMOMENT method on the given SBML
# inputs:   - path to the input SBML model
#           - name of the output SBML (protein-constraint-enhanced model)
#           - project folder containing reaction<->kcat mapping, protein<->mass mapping and enzyme stoichiometry spreadsheet
#           - project name
#           - excluded reactions for which the pseudometabolite of the protein pool shall not be introduced
#           - type of default kcat selection (can be "mean", "median" or "random")
# output:   - protein-constraint-enhanced version of the input model

from autopacmen.submodules.create_smoment_model_reaction_wise import create_smoment_model_reaction_wise_with_sbml

create_smoment_model_reaction_wise_with_sbml(
    input_sbml_path=str(xml_model_dir),
    output_sbml_name=str(output_model_name),
    project_folder=str(input_dir),
    project_name=str(project_name),
    excluded_reactions=[], # reactions can be excluded from the pseudometabolite constrain
    type_of_default_kcat_selection="mean"
)

***
Reaction: MAR03905
Forward kcat: 2.569637839063218
Reverse kcat: 2.569637839063218

***
Reaction: MAR03907
Forward kcat: 3.944748461345614
Reverse kcat: 3.944748461345614

***
Reaction: MAR04097
Forward kcat: 107.02857142857142
Reverse kcat: 107.02857142857142

***
Reaction: MAR04099
Forward kcat: 107.02857142857142
Reverse kcat: 107.02857142857142

***
Reaction: MAR04108
Forward kcat: 107.02857142857142
Reverse kcat: 107.02857142857142

***
Reaction: MAR04133
Forward kcat: 107.02857142857142
Reverse kcat: 107.02857142857142

***
Reaction: MAR04281
Forward kcat: 220.87253564705884
Reverse kcat: 220.87253564705884

***
Reaction: MAR04388
Forward kcat: 220.87253564705884
Reverse kcat: 220.87253564705884

***
Reaction: MAR04283
Forward kcat: 8.2
Reverse kcat: 8.2

***
Reaction: MAR08357
Forward kcat: 18.170115717216483
Reverse kcat: 18.170115717216483

***
Reaction: MAR04379
Forward kcat: 241.21004633333334
Reverse kcat: 241.21004633333334

***
Reaction: MAR04301
Forward kcat: 241.210

c:\Users\phili\anaconda3\envs\context_GEM\Lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


ENSG00000103253
ENSG00000103253
ENSG00000185875


In [ ]:
# Applies the sMOMENT method on the given SBML
# inputs:   - path to the input SBML model
#           - name of the output SBML (protein-constraint-enhanced model)
#           - project folder containing reaction<->kcat mapping, protein<->mass mapping and enzyme stoichiometry spreadsheet
#           - project name
#           - excluded reactions for which the pseudometabolite of the protein pool shall not be introduced
#           - type of default kcat selection (can be "mean", "median" or "random")
# output:   - protein-constraint-enhanced version of the input model

from autopacmen.submodules.create_smoment_model_reaction_wise import create_smoment_model_reaction_wise_with_sbml

create_smoment_model_reaction_wise_with_sbml(
    input_sbml_path=str(xml_model_dir),
    output_sbml_name=str(output_model_name),
    project_folder=str(input_dir),
    project_name=str(project_name),
    excluded_reactions=[], # reactions can be excluded from the pseudometabolite constrain
    type_of_default_kcat_selection="mean"
)

# *** Important Note ***
# Because of the size of the Human1 model, this step might take days to compute on a standard computer. This is why computation on 
# a cluster is recommended. For this, it is easiest to execute the script from the console using this command:

# python modeling_create_smoment_model.py –-input_sbml_path <project_directory>/thesis_scrips/models/<model_sbml_name>.xml 
# --output_sbml_name <output_model_name>.xml --project_folder <project_directory>/thesis_scripts/datasets/autopacmen/autopacmen_input/ 
# --project_name <project_name> --excluded_reactions <semicolon seperated list of reactions to exclude> --type_of_default_kcat_selection 'mean'